<a href="https://colab.research.google.com/github/bob8dod/DeepLearning-with-PyTorch/blob/main/assignment_3_titanic_with_mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# 필요 라이브러리 임포트 (데이터 분석을 위한 라이브러리들)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# 시드 고정 (이후에도 똑같은 결과가 나오게 하기 위함)
seed = 3
np.random.seed(seed)

In [ ]:
# data 로드
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')
submission = pd.read_csv('../input/titanic/gender_submission.csv')

가장 먼저 MLP를 통한 survived 예측 전, 예측에 사용될 특성과 버릴 특성 및 특성을 생성해야 될 부분이 있나 EDA를 통해 분석해 보았습니다.

In [ ]:
# 데이터 확인
print('Train set:',df_train.shape)
print('Test set:',df_test.shape, end='\n\n')
print('------------[Train Data Infomation]-------------')
print(df_train.info(),end='\n\n')
print('------------[Test Data Infomation]-------------')
print(df_test.info())

!해당 column들을 확인하고 trian과 test data 모두 Dtype이 동일하다는 것을 확인!  

!Age,Fare,Cabin,Embarked 같은 경우 nan 값이 존재한다는 것을 확인 !   
  
**변수설명**  
PassengerId : 각 승객의 고유 번호  
Survived : 생존 여부(예측 해야될 특성) => (0 = 사망 / -1 = 생존)  
Pclass : 객실 등급 (승객의 사회적, 경제적 지위 반영) => (1st = Upper / 2nd = Middle / 3rd = Lower)  
Name : 이름  
Sex : 성별  
Age : 나이  
SibSp : 동반한 Sibling(형제자매)와 Spouse(배우자)의 수  
Parch : 동반한 Parent(부모) Child(자식)의 수  
Ticket : 티켓의 고유넘버  
Fare : 티켓의 요금  
Cabin : 객실 번호  
Embarked : 승선한 항 (C = Cherbourg, Q = Queenstown, S = Southampton)

column들을 확인 후 각 특성들이 결과(생존 여부)와 관련이 있는지 시각적(파이 차트, 바 차트)으로 확인해 보았습니다.

In [ ]:
def pie_chart(feature): #'Sex'를 기준으로 설명
    feature_ratio = df_train[feature].value_counts(sort=False) # {male:#, female:#}
    feature_size = feature_ratio.size # 2 _ (male,female)
    feature_index = feature_ratio.index 
    # Index(['female', 'male'], dtype='object')
    survived = df_train[df_train['Survived'] == 1][feature].value_counts() 
    #해당 feature에서 생존한 사람들에 대해서 새로운 list 생성 -> {male:#, female:#}
    dead = df_train[df_train['Survived'] == 0][feature].value_counts() 
    #해당 feature에서 사망한 사람들에 대해서 새로운 list 생성 -> {male:#, female:#}

    #Total ratio
    plt.plot(aspect='auto')
    plt.pie(feature_ratio, labels=feature_index, autopct='%1.1f%%')
    plt.title(feature + "'s total ratio")
    plt.show()
  
  #sub ratio
    for i, index in enumerate(feature_index):
        plt.subplot(1, feature_size + 1, i + 1, aspect='equal')
        plt.pie([survived[index], dead[index]], labels=['O', 'X'], autopct='%1.1f%%')
        plt.title(str(index) + "'s O/X ratio")
    plt.show()

In [ ]:
def bar_chart(feature):
    total = df_train[feature].value_counts()
    survived = df_train[df_train['Survived']==1][feature].value_counts()
    dead = df_train[df_train['Survived']==0][feature].value_counts()
    df = pd.DataFrame([total,survived,dead])
    df.index = ['Total','Survived','Dead']
    df.plot(kind='bar',stacked=True, figsize=(10,5),alpha=0.75, rot=0,
            title="{}'s ratio".format(feature))

In [ ]:
# pie_chart ('Pclass', 'Sex', 'Embarked')
for col in ['Pclass', 'Sex', 'Embarked']:
    pie_chart(col)

시각적으로 표현한 파이 차트를 통해  
성별이 여자일 수록 생존 확률이 높고  
객실 등급이 높아질 수록 생존 확률이 높다는 것을 확인할 수 있습니다.  
추가적으로 C에서 승선한 승객이 S 나 Q에서 승선한 승객보다 생존 확률이 더 높다는 것을 확인할 수 있습니다.
즉, Pclass 와 Sex, Embarked 특성은 생존 여부에 영향을 강하게 끼치는 것을 알 수 있습니다.

In [ ]:
for col in ['SibSp', 'Parch']:
    bar_chart(col)

해당 차트를 통해서 혼자 배에 탑승한 경우 생존확률이 다른 경우보다 낮단느 것을 확인할 수 있습니다.  
혼자 타지 않은 탑승객인 경우 동반 탑승객의 수와 상관없이 크게 다르지 않은 생존 비율을 보입니다.  
이를 통해 혼자 탔냐 여럿이 탔냐, 2가지로 구분하여 특성을 사용해도 되겠다고 판단이 가능합니다.

In [ ]:
# Age 같은 경우 구간을 설정하여 판단
for dataset in [df_train, df_test]:
    dataset[dataset['Age'] <= 0]['Age'] = 0
    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True) 
    # inplace=True, 채우고 저장함
    dataset['Age'] = dataset['Age'].astype(int)
    dataset['AgeBand'] = pd.cut(df_train['Age'],8) # 8개의 구간으로 나눔
df_train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean()

Age 는 유아일 경우가 생존 확률이 높고  
노인일 수록 생존확률이 낮다는 것을 확인 가능

이외의 특성들(PassengerId, Name, Ticket, Cabin)에서 의미를 찾아보겠습니다.  
먼저 PassengerId 같은 경우 말그대로 Id 이기에 제외해도 되겠다고 생각했고,    
Name 같은 경우 Age, SibSp, Parch 과 상관이 있다 판단하여 제거했습니다.   
마지막으로 Ticket 이나 Cabin 같은 경우도 Pclass와 관련된 특성이라 판단하여 제거하였습니다.  

**데이터 전처리**  
위의 분석을 바탕으로 버릴 특성들은 버리고  
기존의 특성의 구간을 나누어주거나 string -> int 로, 필요한 특성이 있다면 추가하는 전처리 작업을 진행해 보았습니다,

In [ ]:
df_train['Embarked'].value_counts(dropna=False) #dropna =Flase_nan값까지 세어줌...
# Embarked 같은 경우 Nan 값 존재 확인

In [ ]:
# 버릴 특성들 drop 해주고 Sex, Embarked 처리
df_train = df_train.drop(['PassengerId','Name','Ticket',
                          'Cabin','AgeBand','Fare'], axis=1)
df_train['Sex'] = df_train['Sex'].astype(str)
df_train['Embarked'] = df_train['Embarked'].fillna('S') #NaN을 최빈값인 S로 대체
df_train['Embarked'] = df_train['Embarked'].astype(str)

df_test = df_test.drop(['PassengerId','Name','Ticket',
                        'Cabin','AgeBand','Fare'], axis=1)
df_test['Sex'] = df_test['Sex'].astype(str)
df_test['Embarked'] = df_test['Embarked'].fillna('S') #NaN을 최빈값인 S로 대체
df_test['Embarked'] = df_test['Embarked'].astype(str)

In [ ]:
df_train.head(3)

In [ ]:
# Age 와 SibSp, Parch 구간 설정
for dataset in [df_train, df_test]:
    # Age
    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True)
    dataset.loc[dataset['Age'] <= 10, 'Age'] = 0 
    #전체 sample중에서 Age가 10보다 작은 사람들의 Age 값을 0으로 할당
    dataset.loc[(dataset['Age'] > 10) & (dataset['Age'] <= 20), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 20) & (dataset['Age'] <= 30), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 30) & (dataset['Age'] <= 40), 'Age'] = 3
    dataset.loc[(dataset['Age'] > 40) & (dataset['Age'] <= 50), 'Age'] = 4
    dataset.loc[(dataset['Age'] > 50) & (dataset['Age'] <= 60), 'Age'] = 5
    dataset.loc[(dataset['Age'] > 60) , 'Age'] = 6
    dataset['Age'] = dataset['Age'].map( { 0: 'Child',  1: 'Young', 2: 'Middle', 
                                          3: 'Prime', 4: 'Old', 
                                          5: 'Older', 6: 'Oldest'} ).astype(str) #classification 진행
    # Sibsp, Parch
    dataset.loc[(dataset['SibSp'] <= 0) & (dataset['Parch'] <= 0), 'Family'] = 0
    dataset.loc[(dataset['SibSp'] > 0) | (dataset['Parch'] > 0), 'Family'] = 1
    dataset['Family'] = dataset['Family'].astype(int)
    dataset.drop(['SibSp','Parch'],axis=1,inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
# str -> int Encoding 진행 (sklearn 사용)
from sklearn.preprocessing import LabelEncoder
for col in ['Sex', 'Age', 'Embarked']:
    le = LabelEncoder()
    df_train[col] = le.fit_transform(df_train[col])
    df_test[col] = le.transform(df_test[col])

In [ ]:
df_train.head()

In [ ]:
df_test.head()

학습

In [ ]:
# 학습을 위해 종속변수 분리 및 학습 및 검증 데이터 분리
from sklearn.model_selection import train_test_split
y = df_train['Survived']
df_train = df_train.drop(['Survived'],axis=1)
x_train, x_test, y_train, y_test = train_test_split(df_train, y, 
                                                    test_size=0.3, 
                                                    random_state=3)

학습 MLP 사용

In [ ]:
# 실습을 위해 파이토치의 도구들을 임포트하는 기본 셋팅을 진행
import torch
import torch.nn as nn
from torch import optim
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [ ]:
X_train = torch.FloatTensor(np.array(x_train)).to(device)
Y_train = torch.FloatTensor(np.array(y_train)).unsqueeze(1).to(device)
X_test = torch.FloatTensor(np.array(x_test)).to(device)
Y_test = torch.FloatTensor(np.array(y_test)).unsqueeze(1).to(device)

In [ ]:
model = nn.Sequential(
    nn.Linear(5, 16), # input_layer = 5, hidden_layer1 = 16
    nn.ReLU(),#     nn.Dropout(0.1),
    nn.Linear(16, 8), # hidden_layer2 = 16, hidden_layer3 = 8
    nn.ReLU(),#     nn.Dropout(0.1),
    nn.Linear(8, 1), # hidden_layer3 = 8, output_layer = 1
    nn.Sigmoid()
).to(device)

In [ ]:
criterion = torch.nn.BCELoss().to(device) 
# 이진 분류에서 사용하는 크로스엔트로피 함수
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5) 
# Adam optimizer 사용 learning rate는 0.01, 가중치 규제(Regularization) 적용
# optimizer = torch.optim.SGD(model.parameters(), lr=1)

In [ ]:
for epoch in range(1001):
    model.train()  # 신경망을 학습 모드로 전환
    optimizer.zero_grad() # 경사를 0으로 초기화
    y_pred = model(X_train) # forward 연산, 데이터를 입력하고 출력을 계산
    loss = criterion(y_pred, Y_train) # 출력과 훈련 데이터 정답 간의 오차를 계산
    loss.backward() # 오차를 역전파 계산
    optimizer.step() # 역전파 계산한 값으로 가중치를 수정

    # epoch이 100단위로 끊길 때마다 cost, 학습 accuracy, 검증 accuracy 확인
    if epoch % 100 == 0:
        model.eval()
        with torch.no_grad():
            hypothesis1 = model(X_train)
            predicted1 = (hypothesis1 > 0.5).float()
            accuracy1 = (predicted1 == Y_train).float().mean()
            
            hypothesis2 = model(X_test)
            predicted2 = (hypothesis2 > 0.5).float()
            accuracy2 = (predicted2 == Y_test).float().mean()
        
        print('Epoch {:5d}/{} Cost: {:.6f} Train_Acc: {:.4f} Val_Acc: {:.4f}'.format(
                epoch, 1000, loss.item(), accuracy1.item(), accuracy2.item()))

In [ ]:
# 데이터 
test = torch.FloatTensor(np.array(df_test)).to(device)
with torch.no_grad():
    hypothesis = model(test)
    predicted = (hypothesis > 0.5).float()
pred = pd.DataFrame(np.array(predicted).astype(int))

submission['Survived'] = pred
submission.to_csv('submission.csv', index=False)